# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [2]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [12]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [13]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [14]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [15]:
# ... ENTER YOUR CODE HERE ...

import numpy as np
from numpy.testing import assert_allclose

def jacobi_iteration(A, b, n_iter=50):
    # Separate the diagonal part
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    
    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B
    c = invD @ b
    
    # Jacobi iteration
    x = np.ones_like(b)
    for _ in range(n_iter):
        x = BB @ x + c
    
    return x

# Construct the matrix and r.h.s. for the Jacobi iteration
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# Sanity checks
assert_allclose(-B + D, A)

# "Ground truth" solution using a direct method
xx = np.linalg.solve(A, b)
np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

# Check the norm of BB
norm_BB = np.linalg.norm(BB)
print(f"Norm of BB: {norm_BB}")

# Perform Jacobi iteration
n_iter = 50
x_jacobi = jacobi_iteration(A, b, n_iter)

# Check the result
residual = A @ x_jacobi - b
print(f"Residual after {n_iter} Jacobi iterations:\n{residual}")


Norm of BB: 0.36436161983015336
Residual after 50 Jacobi iterations:
[ 1.11022302e-16  0.00000000e+00 -2.22044605e-16 -1.11022302e-16
  1.11022302e-16  0.00000000e+00 -2.42861287e-17  0.00000000e+00
 -2.77555756e-17  1.11022302e-16]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [10]:
# ... ENTER YOUR CODE HERE ...

import numpy as np
from numpy.testing import assert_allclose

def seidel_iteration(A, b, n_iter=50):
    # Separate the lower and upper triangular parts
    L = np.tril(A)
    U = A - L
    
    # Invert the lower triangular part
    invL = np.linalg.inv(L)
    
    # Seidel's iteration
    x = np.ones_like(b)
    for _ in range(n_iter):
        x = np.dot(invL, b - np.dot(U, x))
    
    return x

# Test on a random matrix
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# "Ground truth" solution using a direct method
xx = np.linalg.solve(A, b)
np.testing.assert_allclose(A@xx, b)

# Perform Seidel's iteration
n_iter = 50
x_seidel = seidel_iteration(A, b, n_iter)

# Check the result
residual = A @ x_seidel - b
print(f"Residual after {n_iter} Seidel iterations:\n{residual}")

# Check the norm of the iteration matrix
iteration_matrix = np.dot(np.linalg.inv(np.tril(A)), A - np.tril(A))
norm_iteration_matrix = np.linalg.norm(iteration_matrix)
print(f"Norm of the iteration matrix: {norm_iteration_matrix}")



Residual after 50 Seidel iterations:
[ 0.00000000e+00 -1.11022302e-16 -2.22044605e-16 -1.11022302e-16
  1.11022302e-16  0.00000000e+00 -3.46944695e-18  0.00000000e+00
  2.77555756e-17  0.00000000e+00]
Norm of the iteration matrix: 0.26515087068954285


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [11]:
# ... ENTER YOUR CODE HERE ...

import numpy as np
from numpy.testing import assert_allclose

def minimum_residual_scheme(A, b, n_iter=50):
    x = np.ones_like(b)
    r = b - np.dot(A, x)
    
    for _ in range(n_iter):
        alpha = np.dot(r, A @ r) / np.dot(A @ r, A @ r)
        x = x + alpha * r
        r = b - np.dot(A, x)
    
    return x

# Test on a random matrix
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# "Ground truth" solution using a direct method
xx = np.linalg.solve(A, b)
np.testing.assert_allclose(A@xx, b)

# Perform Minimum Residual Scheme
n_iter = 50
x_min_residual = minimum_residual_scheme(A, b, n_iter)

# Check the result
residual = A @ x_min_residual - b
deviation = x_min_residual - xx
print(f"Residual after {n_iter} iterations:\n{residual}")
print(f"Deviation from the ground truth solution:\n{deviation}")

# Study the variation of the iteration parameter
iteration_parameters = []
x = np.ones_like(b)
r = b - np.dot(A, x)
for _ in range(n_iter):
    alpha = np.dot(r, A @ r) / np.dot(A @ r, A @ r)
    iteration_parameters.append(alpha)
    x = x + alpha * r
    r = b - np.dot(A, x)

print(f"Iteration parameters:\n{iteration_parameters}")


Residual after 50 iterations:
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.46944695e-18 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Deviation from the ground truth solution:
[ 0.00000000e+00  0.00000000e+00  6.93889390e-18  6.93889390e-18
 -1.38777878e-17 -1.51788304e-18  0.00000000e+00  6.93889390e-18
 -1.73472348e-18  1.38777878e-17]
Iteration parameters:
[0.04945809191591971, 0.06632815899696644, 0.05646528371934242, 0.0591368614483341, 0.056149365850671804, 0.05962056630692012, 0.05532125962259367, 0.06019139542969565, 0.0547265196248599, 0.060918354717434245, 0.05478879425690857, 0.061543172800129886, 0.0551772119379602, 0.0616212506868999, 0.0551452969997398, 0.06170111966684046, 0.05472579796901509, 0.06212187143489488, 0.054009338786712234, 0.06441861709831852, 0.05443303564328216, 0.06011645923749508, 0.06271150857694172, 0.06413230252109958, 0.06413230252109958, 0.06413230252109958, 0.06413230252109958, 0.06413230252109958, 